In [ ]:
import pydicom as dicom
import matplotlib.pylab as plt
import numpy as np
import os

import cv2
from natsort import natsorted
from skimage.registration import phase_cross_correlation
from scipy import ndimage as scp
from tqdm import tqdm
import pickle

import ants.registration as ants_register
import ants

In [ ]:
# after drop 1
# Loading the dataset
path = '/Users/akapatil/Documents/OCT/pig_eyeball/after_apply_drop_1/pic1/'
pic_paths = []
for i in os.listdir(path):
    if i.endswith('.dcm') or  i.endswith('.DCM') or i.endswith('.PNG'):
        pic_paths.append(i)
pic_paths = np.array(natsorted(pic_paths))[range(0,len(pic_paths),2)]

# reading all te dicom images
pics_without_line = []
for i in tqdm(pic_paths):
    aa = dicom.dcmread(path+i).pixel_array
    pics_without_line.append(aa.copy())

# Breaking down the dataset into parts
# Need to do it manually after checking registration
pics_without_line = np.array(pics_without_line)
# data_after_drop1_50 = pics_without_line[:50]
# data_after_drop1_90 = pics_without_line[50:100]
# data_after_drop1_100 = pics_without_line[100:200]
# data_after_drop1_500 = pics_without_line[200:500]
# data_after_drop1_1250 = pics_without_line[500:1250]
# data_after_drop1_1650 = pics_without_line[1250:1650]
# data_after_drop1_remain = pics_without_line[1650:]

# Try to delete unnecessary/repeated variables to keep memory usage low
del pics_without_line

In [ ]:

# Performing Phase registration first to make affine more efficient

def phase(data):
    n = data.shape[0]//2
    for i in tqdm(range(data.shape[0])):
        coords = phase_cross_correlation(data[n],data[i],normalization=None)[0]
        data[i] = scp.shift(data[i],shift = (coords[0],coords[1]),mode='constant',order=0)
    return data

data_after_drop1_50 = phase(data_after_drop1_50)
data_after_drop1_90 = phase(data_after_drop1_90)
data_after_drop1_100 = phase(data_after_drop1_100)
data_after_drop1_500 = phase(data_after_drop1_500)
data_after_drop1_1250 = phase(data_after_drop1_1250)
data_after_drop1_1650 = phase(data_after_drop1_1650)
data_after_drop1_remain = phase(data_after_drop1_remain)


In [ ]:
# define mask for registration
# you can select any patch in the data you want, the smaller the more faster algorithm is
# I have not tried different patches, so you can definitely try more, also you can use different patches for different datasets

moving_mask = np.zeros_like(data_after_drop1_100[0])
moving_mask[300:450,400:600] = 1

In [ ]:

# Main Registration fucntion that registers the individual chunks
def ants_reg(stat,mov):
    ants1 = ants.from_numpy(mov.astype(np.float64))
    ants2 = ants.from_numpy(stat.astype(np.float64))
    mov_mask = ants.from_numpy(moving_mask.astype(np.float64))
    reg = ants_register(ants2,ants1,type_of_transform = 'Translation',moving_mask = mov_mask,mask = mov_mask,mask_all_stages=True)
    reg_img = ants.apply_transforms(ants2, ants1, reg['fwdtransforms'])
    return reg_img.numpy()

# Registration fucntion that calculates the mapping between two consecutuive chunks
def ants_reg_mapping(stat,mov):
    ants1 = ants.from_numpy(mov.astype(np.float64))
    ants2 = ants.from_numpy(stat.astype(np.float64))
    mov_mask = ants.from_numpy(moving_mask.astype(np.float64))
    reg = ants_register(ants2,ants1,type_of_transform = 'Translation',moving_mask = mov_mask,mask = mov_mask,mask_all_stages=True)
    # reg_img = ants.apply_transforms(ants2, ants1, reg['fwdtransforms'])
    return reg['fwdtransforms']

# Registration fucntion that maps the mapping to the previous chunk
def ants_reg_translate(stat,mov,maping):
    ants1 = ants.from_numpy(mov.astype(np.float64))
    ants2 = ants.from_numpy(stat.astype(np.float64))
    mov_mask = ants.from_numpy(moving_mask.astype(np.float64))
    # reg = ants_register(ants2,ants1,type_of_transform = 'Translation',moving_mask = mov_mask,mask = mov_mask,mask_all_stages=True)
    reg_img = ants.apply_transforms(ants2, ants1, maping,interpolator='nearestNeighbor')
    return reg_img.numpy()

# running the registration for a chunk
def reg(data):
    reg_after_drop1 = []
    n = data.shape[0]//2
    for i in tqdm(range(0,data.shape[0])):
        regis = ants_reg(data[n],data[i])
        reg_after_drop1.append(regis)
    reg_after_drop1 = np.array(reg_after_drop1)
    return reg_after_drop1
data_after_drop1_50 = reg(data_after_drop1_50)
data_after_drop1_90 = reg(data_after_drop1_90)
data_after_drop1_100 = reg(data_after_drop1_100)
data_after_drop1_500 = reg(data_after_drop1_500)
data_after_drop1_1250 = reg(data_after_drop1_1250)
data_after_drop1_1650 = reg(data_after_drop1_1650)
data_after_drop1_remain = reg(data_after_drop1_remain)


In [ ]:

# Registering two consecutuive chunks by calculating the mapping between last image of prev chunk and the first image of next chunk
# Then apply the mapping to all images in the prev chunk.

test00 = []
map00 = ants_reg_mapping(data_after_drop1_90[0],data_after_drop1_50[-1])
for i in tqdm(range(data_after_drop1_50.shape[0])):
    test00.append(ants_reg_translate(data_after_drop1_90[0],data_after_drop1_50[i],map00))
for i in tqdm(range(data_after_drop1_90.shape[0])):
    test00.append(data_after_drop1_90[i])


test0 = []
map0 = ants_reg_mapping(data_after_drop1_100[0],test00[-1])
for i in tqdm(range(len(test00))):
    test0.append(ants_reg_translate(data_after_drop1_100[0],test00[i],map0))
for i in tqdm(range(data_after_drop1_100.shape[0])):
    test0.append(data_after_drop1_100[i])

del test00


test1 = []
map1 = ants_reg_mapping(data_after_drop1_500[0],test0[-1])
for i in tqdm(range(len(test0))):
    test1.append(ants_reg_translate(data_after_drop1_500[0],test0[i],map1))
for i in tqdm(range(data_after_drop1_500.shape[0])):
    test1.append(data_after_drop1_500[i])

del test0

test2=[]
map2 = ants_reg_mapping(data_after_drop1_1250[0],test1[-1])
for i in tqdm(range(len(test1))):
    test2.append(ants_reg_translate(data_after_drop1_1250[0],test1[i],map2))
for i in tqdm(range(data_after_drop1_1250.shape[0])):
    test2.append(data_after_drop1_1250[i])

del test1

test3=[]
map3 = ants_reg_mapping(data_after_drop1_1650[0],test2[-1])
for i in tqdm(range(len(test2))):
    test3.append(ants_reg_translate(data_after_drop1_1650[0],test2[i],map3))
for i in tqdm(range(data_after_drop1_1650.shape[0])):
    test3.append(data_after_drop1_1650[i])

del test2

test4 = []
map4 = ants_reg_mapping(data_after_drop1_remain[0],test3[-1])
for i in tqdm(range(len(test3))):
    test4.append(ants_reg_translate(data_after_drop1_remain[0],test3[i],map4))
for i in tqdm(range(data_after_drop1_remain.shape[0])):
    test4.append(data_after_drop1_remain[i])

del test3

In [ ]:

# Save the images, make sure you save them uint16

# os.mkdir('2D/2D_timelapse_postsolution/registered/scan2/')
for i,j in tqdm(enumerate(test4)):
    cv2.imwrite('/Users/akapatil/Documents/OCT/pig_eyeball/test/'+f'frame_test{i}.PNG',j.astype(np.uint16))